In [1]:
import numpy as np 
import matplotlib as mp
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.contrib.layers.python.layers import batch_norm
import pandas as pd
from skimage import img_as_float, img_as_ubyte, io
import os
from Tkinter import *
from PIL import ImageTk, Image
from skimage.draw import polygon_perimeter
from skimage import transform, exposure

In [2]:
data = None
mean_img = None

# Hyper Params
TOTAL_PATCHES = None
NUM_IMAGES = None
PATCHES_PER_IMAGE = None
PATCH_DIM = 31
BATCH_SIZE = 1
LEARNING_RATE = 5e-4
TRAINING_PROP = 0.9
MAX_STEPS = 125
CKPT_STEP = 40
LOSS_STEP = 2
KEEP_PROB = 0.6
NUM_CLASSES = 2
FCHU1 = 256               # Fully connected layer 1 hidden units
MODEL_NAME = '19'
MODEL_PATH = '../../Data/models/model19/model.ckpt-5399'

In [3]:
def batch_norm_layer(x,train_phase,scope_bn):
    """Adds a Batch Normalization layer from http://arxiv.org/abs/1502.03167.
        "Batch Normalization: Accelerating Deep Network Training by Reducing
        Internal Covariate Shift"
        Sergey Ioffe, Christian Szegedy
      Can be used as a normalizer function for conv2d and fully_connected.
    """
    
    bn_train = batch_norm(x, decay=0.999, center=True, scale=True, updates_collections=None,
    is_training=True, reuse=None, # is this right?
    trainable=True, scope=scope_bn)
    
    bn_inference = batch_norm(x, decay=0.999, center=True, scale=True, updates_collections=None,
    is_training=False, reuse=True, # is this right?
    trainable=True, scope=scope_bn)
    
    z = tf.cond(train_phase, lambda: bn_train, lambda: bn_inference)
    return z


In [4]:
def placeholder_inputs(batch_size):
    """Generate placeholder variables to represent the input tensors.
    
    These placeholders are used as inputs by the rest of the model building
    code and will be fed from the downloaded data in the .run() loop, below.
    Args:
        batch_size: The batch size will be baked into both placeholders.
    Returns:
        images_placeholder: Images placeholder.
        labels_placeholder: Labels placeholder.
    """
    # Note that the shapes of the placeholders match the shapes of the full
    # image and label tensors, except the first dimension is now batch_size
    # rather than the full size of the train or test data sets.
    images_placeholder = tf.placeholder(tf.float32, shape=(batch_size, PATCH_DIM**2*3))
    labels_placeholder = tf.placeholder(tf.int32, shape=(batch_size, NUM_CLASSES))
    return images_placeholder, labels_placeholder

In [5]:
#data = pd.read_pickle('../../Data/mean_normalised_df_no_class_bias.pkl') 
mean_img = pd.read_pickle('../../Data/mean_img_no_class_bias.pkl')

#TOTAL_PATCHES = len(data)
#NUM_IMAGES = len(train)
#PATCHES_PER_IMAGE = TOTAL_PATCHES/NUM_IMAGES
PATCH_DIM = 31
    

images, labels_placeholder = placeholder_inputs(BATCH_SIZE)
keep_prob = tf.placeholder(tf.float32)
train_phase = tf.placeholder(tf.bool)

In [6]:
#---------------------------Inference------------------------------------
with tf.variable_scope('h_conv1') as scope:
        weights = tf.get_variable('weights', shape=[4, 4, 3, 64], 
                                  initializer=tf.contrib.layers.xavier_initializer_conv2d())
        biases = tf.get_variable('biases', shape=[64], initializer=tf.constant_initializer(0.05))
        
        x_image = tf.reshape(images, [-1,PATCH_DIM,PATCH_DIM,3])
        x_image_bn = batch_norm_layer(x_image, train_phase, scope.name)
        z = tf.nn.conv2d(x_image_bn, weights, strides=[1, 1, 1, 1], padding='VALID') + biases
        h_conv1 = tf.nn.relu(z, name=scope.name)
        
with tf.variable_scope('h_conv2') as scope:
        weights = tf.get_variable('weights', shape=[4, 4, 64, 64], 
                                  initializer=tf.contrib.layers.xavier_initializer_conv2d())
        biases = tf.get_variable('biases', shape=[64], initializer=tf.constant_initializer(0.05))
        h_conv1_bn = batch_norm_layer(h_conv1, train_phase, scope.name)
        z = tf.nn.conv2d(h_conv1_bn, weights, strides=[1, 1, 1, 1], padding='SAME')+biases
        h_conv2 = tf.nn.relu(z, name=scope.name)
    
h_pool1 = tf.nn.max_pool(h_conv2, ksize=[1, 2, 2, 1],
                        strides=[1, 2, 2, 1], padding='SAME', name='h_pool1')
    
with tf.variable_scope('h_conv3') as scope:
        weights = tf.get_variable('weights', shape=[4, 4, 64, 64], 
                                  initializer=tf.contrib.layers.xavier_initializer_conv2d())
        biases = tf.get_variable('biases', shape=[64], initializer=tf.constant_initializer(0.05))
        h_pool1_bn = batch_norm_layer(h_pool1, train_phase, scope.name)
        z = tf.nn.conv2d(h_pool1_bn, weights, strides=[1, 1, 1, 1], padding='SAME')+biases
        h_conv3 = tf.nn.relu(z, name=scope.name)
        
h_pool2 = tf.nn.max_pool(h_conv3, ksize=[1, 2, 2, 1],
                        strides=[1, 2, 2, 1], padding='SAME', name='h_pool2')    
    
with tf.variable_scope('h_fc1') as scope:
        weights = tf.get_variable('weights', shape=[7**2*64, FCHU1], 
                                  initializer=tf.contrib.layers.xavier_initializer())
        biases = tf.get_variable('biases', shape=[FCHU1], initializer=tf.constant_initializer(0.05))
        h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])
        #h_pool2_flat_bn = batch_norm_layer(h_pool2_flat, train_phase, scope.name)
        h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, weights) + biases, name = 'h_fc1')
        h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)
        
with tf.variable_scope('h_fc2') as scope:
        weights = tf.get_variable('weights', shape=[FCHU1, NUM_CLASSES], 
                                  initializer=tf.contrib.layers.xavier_initializer())
        biases = tf.get_variable('biases', shape=[NUM_CLASSES])
        #h_fc1_drop_bn = batch_norm_layer(h_fc1, train_phase, scope.name)
        logits = (tf.matmul(h_fc1_drop, weights) + biases)
#--------------------------------------------------------------------------------
        

In [7]:
saver = tf.train.Saver()

# Create a session for running Ops on the Graph.
sess = tf.Session()
saver.restore(sess, MODEL_PATH)


In [8]:
def getActivations(layer,stimuli,l_name,root_path):
    
    layer_path = os.path.join(root_path,l_name)
    if not os.path.exists(layer_path):
        os.mkdir(layer_path)
    
        
    units = layer.eval(session=sess,
                       feed_dict={images:stimuli, keep_prob:1.0, train_phase:False})
    plot_save_NNFilter(units,l_name,root_path)
    
def plot_save_NNFilter(units,l_name,root_path):
    layer_path = os.path.join(root_path,l_name)
    filters = units.shape[3]
    
    plt.figure(1, figsize=(20,20))
    for i in xrange(0,filters):
        plt.subplot(8,8,i+1)
        plt.title('Filter ' + str(i))
        filter_path = os.path.join(layer_path,'Filter'+str(i)+'.png')
        img_to_save = exposure.rescale_intensity(1.0*units[0,:,:,i])
        img_to_save = transform.resize(img_as_float(img_to_save), (150,150))
        io.imsave(filter_path, img_to_save)
        plt.imshow(units[0,:,:,i], interpolation="nearest", cmap="gray")
    plt.savefig(os.path.join(root_path,l_name+'plot.png'), bbox_inches='tight')

In [9]:
class App:

    def __init__(self, master, path, plot_save_path):

        self.root = master
        frame = Frame(master)
        frame.pack()
        
        self.plot_save_path = os.path.abspath(plot_save_path)
        
        self.main_img_np = io.imread(path)
        pt = [self.main_img_np.shape[0]/2, self.main_img_np.shape[1]/2]
        x = pt[1]
        y = pt[0]
        
        self.main_img_annotated_np = self.main_img_np.copy()
        r,c = self.get_rect(pt)
        self.main_img_annotated_np[r,c] = [64,224,208]
        self.main_img = self.getImageTk(self.main_img_annotated_np)
        self.main_img_label = Label(frame, image = self.main_img)
        self.main_img_label.grid(row=0, column=0, rowspan=4)
        
        h = PATCH_DIM/2
        self.crop_img_np = self.main_img_np[y-h:y+h+1,x-h:x+h+1]
        self.img_to_use = self.crop_img_np.copy()
        self.crop_img_np = transform.resize(self.crop_img_np, (186,186))
        self.crop_img = self.getImageTk(self.crop_img_np)
        self.crop_img_label = Label(frame, image = self.crop_img)
        self.crop_img_label.grid(row=0, column=1, columnspan=2)

        self.viz = Button(
            frame, text="Viz", command=self.visualize
            )
        self.viz.grid(row=3, column=1)
        self.q = Button(
            frame, text="Quit", command=frame.quit
            )
        self.q.grid(row=3, column=2)
        
        self.x_scale = Scale(frame, from_=h, to_=self.main_img_np.shape[1] - h, 
                             label='x-coordinate', orient=HORIZONTAL,
                             command = self.change_crop)
        self.x_scale.set(x)
        self.x_scale.grid(row=1, column=1, columnspan=2)
        
        self.y_scale = Scale(frame, from_=h, to_=self.main_img_np.shape[0] - h, 
                             label='y-coordinate', orient=HORIZONTAL,
                             command = self.change_crop)
        self.y_scale.set(y)
        self.y_scale.grid(row=2, column=1, columnspan=2)

    def getImageTk(self, img):
        img = img_as_ubyte(img)
        i = Image.fromarray(img)
        return ImageTk.PhotoImage(i)
    def visualize(self):
        imageToUse_flat = np.zeros((1,31**2*3), dtype = 'uint8')
        imageToUse_flat[0] = np.array(np.reshape(self.img_to_use,(31**2*3)) + mean_img, dtype='uint8')
        
        if not os.path.exists(self.plot_save_path):
            os.mkdir(self.plot_save_path)
        
        x = self.x_scale.get()
        y = self.y_scale.get()
        imgs_path = os.path.join(self.plot_save_path, 'x'+str(x)+'y'+str(y))
        if not os.path.exists(imgs_path):
            os.mkdir(imgs_path)
        io.imsave(os.path.join(imgs_path,'Original_Patch.png'), self.crop_img_np)
        
        getActivations(h_conv1,imageToUse_flat,'h_conv1', imgs_path)
        getActivations(h_conv2,imageToUse_flat,'h_conv2', imgs_path)
        getActivations(h_conv3,imageToUse_flat,'h_conv3', imgs_path)
        
    def get_rect(self, pt):
        h = PATCH_DIM/2
        x = pt[0]
        y = pt[1]
        rr, cc = polygon_perimeter([x-h, x+h, x+h, x-h],
                                   [y-h, y-h, y+h, y+h],
                                   shape=self.main_img_np.shape, clip=True)
        return rr, cc
    
    def change_crop(self, event):
        x = self.x_scale.get()
        y = self.y_scale.get()
        pt=[y,x]
        
        self.main_img_annotated_np = self.main_img_np.copy()
        r,c = self.get_rect(pt)
        self.main_img_annotated_np[r,c] = [64,224,208]
        main_img = self.getImageTk(self.main_img_annotated_np)
        self.main_img_label.configure(image = main_img)
        self.main_img_label.image = main_img 
        self.main_img = main_img
        
        h = PATCH_DIM/2
        self.crop_img_np = self.main_img_np[y-h:y+h+1,x-h:x+h+1]
        self.img_to_use = self.crop_img_np.copy()
        self.crop_img_np = transform.resize(self.crop_img_np, (186,186))
        crop_img = self.getImageTk(self.crop_img_np)
        self.crop_img_label.configure(image=crop_img)
        self.crop_img_label.image = crop_img
        self.crop_img = crop_img
        

In [10]:
path = '../../Data/DRIVE/training/images/21_training.tif'
image_save_path = 'images/'
root = Tk()
app = App(root, path, image_save_path)

root.mainloop()
root.destroy() # optional; see description below

/home/ankush/anaconda2/lib/python2.7/site-packages/skimage/util/dtype.py:110: UserWarning: Possible precision loss when converting from float64 to uint8
  "%s to %s" % (dtypeobj_in, dtypeobj))
/home/ankush/anaconda2/lib/python2.7/site-packages/skimage/util/dtype.py:110: UserWarning: Possible precision loss when converting from float64 to uint16
  "%s to %s" % (dtypeobj_in, dtypeobj))
/home/ankush/anaconda2/lib/python2.7/site-packages/skimage/io/_io.py:132: UserWarning: /home/ankush/Projects/DR_Segmentation/Code/Visualizations/images/x70y292/h_conv1/Filter6.png is a low contrast image
  warn('%s is a low contrast image' % fname)
/home/ankush/anaconda2/lib/python2.7/site-packages/numpy/lib/function_base.py:3823: RuntimeWarning: Invalid value encountered in percentile
  RuntimeWarning)
/home/ankush/anaconda2/lib/python2.7/site-packages/skimage/io/_io.py:132: UserWarning: /home/ankush/Projects/DR_Segmentation/Code/Visualizations/images/x70y292/h_conv1/Filter31.png is a low contrast image
 